## Neural Net - 3 Second Features

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [7]:
## Dropping / cleaning data


df_3_sec = pd.read_csv("../data/features_3_sec.csv")


print("Shape all data:", df_3_sec.shape)
# print(df_30_sec.head())

# Encode Labels
le = LabelEncoder()
df_3_sec['label_encoded'] = le.fit_transform(df_3_sec['label'])


print("\nCategories Encoded:")
for i, category in enumerate(le.classes_):
    print(f"{category}: {i}")
    
## Splitting / Scaling data 
y = df_3_sec["label_encoded"].to_numpy()
X = df_3_sec.drop(columns=["filename", "length", "label", "label_encoded"]).to_numpy()

[X_train, X_test, y_train, y_test] = train_test_split(X, y, test_size = .2, random_state=42)

print("\nTrain Size:", len(X_train[:]))
print("Test Size:", len(X_test[:]))

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_test)




Shape all data: (9990, 60)

Categories Encoded:
blues: 0
classical: 1
country: 2
disco: 3
hiphop: 4
jazz: 5
metal: 6
pop: 7
reggae: 8
rock: 9

Train Size: 7992
Test Size: 1998
[[ 2.76810138e-01  1.24248666e+00 -5.88951406e-01 ... -5.65262850e-01
  -3.87971708e-01 -5.04960681e-01]
 [-1.50921822e+00 -6.29612029e-02 -5.90069388e-01 ... -3.63076770e-02
  -1.94199784e+00 -9.70819860e-01]
 [-2.69475846e-01  1.61321684e-01  1.74229852e-03 ... -1.78790286e-01
  -4.25486526e-01 -4.98127262e-01]
 ...
 [ 7.56740266e-01  2.61943788e-01  5.70503346e-01 ...  1.60975327e-01
  -9.62286754e-01 -7.16863122e-02]
 [ 1.43396331e+00 -6.76872934e-01 -6.20973706e-01 ... -4.98466348e-01
   5.06161824e-01  7.42687392e-02]
 [ 3.43792560e-01  1.49588044e+00 -4.69225590e-02 ... -6.02273645e-01
  -1.24274739e+00 -3.70464675e-01]]


## Building Model

In [8]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)



in_size = X_train.shape[1]
print("Input size:", in_size)

class Net(nn.Module):
    def __init__(self, in_size, num_classes=10):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            
            nn.Linear(64, 32),
            nn.ReLU(),

            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.layers(x)

model_NN = Net(in_size)

Input size: 57


## Training Model

In [9]:

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_NN.parameters(), lr=0.05)

num_epochs = 5000

for i in range(num_epochs):
    y_pred = model_NN(X_train_tensor)
    loss = criterion(y_pred, y_train_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    
    if (i+1) % 200 == 0:
        print(f'Epoch [{i+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [200/5000], Loss: 1.7084
Epoch [400/5000], Loss: 1.2663
Epoch [600/5000], Loss: 1.0690
Epoch [800/5000], Loss: 0.9549
Epoch [1000/5000], Loss: 0.8534
Epoch [1200/5000], Loss: 0.7817
Epoch [1400/5000], Loss: 0.7148
Epoch [1600/5000], Loss: 0.6649
Epoch [1800/5000], Loss: 0.6175
Epoch [2000/5000], Loss: 0.5747
Epoch [2200/5000], Loss: 0.5447
Epoch [2400/5000], Loss: 0.5088
Epoch [2600/5000], Loss: 0.4795
Epoch [2800/5000], Loss: 0.4514
Epoch [3000/5000], Loss: 0.4261
Epoch [3200/5000], Loss: 0.3991
Epoch [3400/5000], Loss: 0.3785
Epoch [3600/5000], Loss: 0.3636
Epoch [3800/5000], Loss: 0.3326
Epoch [4000/5000], Loss: 0.3245
Epoch [4200/5000], Loss: 0.3091
Epoch [4400/5000], Loss: 0.2853
Epoch [4600/5000], Loss: 0.2746
Epoch [4800/5000], Loss: 0.2634
Epoch [5000/5000], Loss: 0.2468


## Evaluation

In [10]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

with torch.no_grad():
    y_pred = model_NN(X_test_tensor)
    _, predicted = torch.max(y_pred, dim=1)
    accuracy = (predicted == y_test_tensor).float().mean()
    print(f'Test Accuracy: {accuracy.item():.4f}')

Test Accuracy: 0.8273
